# Data prep

We want to do the same data prep that was done for the 2019 and 2022 radio data here so that it's comparable. To do that without going through all the overly complicated postgres stuff from the first time, we're going to adapt the relevant piece of sql from back then into some python (and sql) this time. Relevant dim data files (stations, shows, etc) are pulled from the original database and in a couple cases extended to accommodate new stations.

In [ ]:
import os
import json
import gzip
import random
import logging

import psycopg2
import numpy as np
import pandas as pd

import matplotlib as mp
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import display
from tqdm.notebook import tqdm

from pandasql import sqldf
pysqldf = lambda s: sqldf(s, globals())

In [ ]:
logger = logging.getLogger(__name__)

fmt = '%(asctime)s : %(levelname)s : %(message)s'
logging.basicConfig(format=fmt, level=logging.INFO)

In [ ]:
os.chdir(os.path.expanduser('~/github/masthesis/'))

In [ ]:
seed = 2969591811

random.seed(seed)
np.random.seed(seed)

# Load data

## Snippet ID info

In [ ]:
with gzip.open('data/paper-round-3/radio/paper-round-3-snippets.csv.gz', 'rt') as f:
    dat = pd.read_csv(f)

snippet_id_start = dat['snippet_id'].max() + 1
del dat

snippet_id_start

## Dim data

In [ ]:
callsign_map = pd.read_csv('data/paper-round-3/metadata/radio/callsign_map.csv', sep=',')
callsign_map = dict(callsign_map[['corpus_callsign', 'station_id']].to_records(index=False).tolist())

show = pd.read_csv('data/paper-round-3/metadata/radio/show.csv', sep='\t')
show_map = dict(show[['name', 'show_id']].to_records(index=False).tolist())

state = pd.read_csv('data/paper-round-3/metadata/radio/state.csv', sep='\t')
state['census_region_5way'] = state['census_region_5way'].str[0]
state_map = dict(state[['postal_code', 'state_id']].to_records(index=False).tolist())

station = pd.read_csv('data/paper-round-3/metadata/radio/station.csv', sep='\t')
station['is_public'] = (station['format'] == 'Public Radio')
station['am_band'] = (station['band'] == 'AM')

## Snippet data

In [ ]:
lines = []

with gzip.open('data/raw/radio/2021.json.gz', 'rt') as f:
    for line in tqdm(f, total=32164520):
        lines += [json.loads(line)]

with gzip.open('data/raw/radio/2022.json.gz', 'rt') as f:
    for line in tqdm(f, total=3970990):
        lines += [json.loads(line)]
        
lines = pd.DataFrame(lines)

## Munge snippet data

In [ ]:
## Date/time variables
lines['start_dt'] = lines['segment_start_global'].apply(lambda s: pd.Timestamp(s, unit='s'))
lines['end_dt'] = lines['segment_end_global'].apply(lambda s: pd.Timestamp(s, unit='s'))
lines['date'] = lines['start_dt'].dt.date.astype(str)
lines['year'] = lines['start_dt'].dt.year

In [ ]:
## Assign state IDs
lines['state_id'] = lines['state'].map(state_map)

In [ ]:
## Assign station IDs
lines['station_id'] = lines['callsign'].map(callsign_map)

In [ ]:
## Assign show IDs
lines['show_name'] = lines['show_name'].str.strip()
lines['show_id'] = lines['show_name'].map(show_map)

In [ ]:
## Drop stations flagged for exclusion
bad_stations = station.loc[station['exclude'] == 't', 'station_id'].tolist()
bad_stations = lines.loc[lines['station_id'].isin(bad_stations)].index
lines.drop(bad_stations, axis=0, inplace=True)

In [ ]:
## To match 2019/2020 data, only snippets with a known show
no_show_id = lines['show_id'].isna()
no_show_id = lines.loc[no_show_id].index
lines.drop(no_show_id, axis=0, inplace=True)
lines['show_id'] = lines['show_id'].astype(int)

In [ ]:
## Drop shows flagged for exclusion
bad_shows = show.loc[show['exclude'] == 't', 'show_id']
bad_shows = lines['show_id'].isin(bad_shows)
bad_shows = lines.loc[bad_shows].index
lines.drop(bad_shows, axis=0, inplace=True)

In [ ]:
## No snippets without transcribed content
bad_content = (lines['content'].isna() | (lines['content'] == ''))
bad_content = lines.loc[bad_content].index
lines.drop(bad_content, axis=0, inplace=True)

In [ ]:
## Assign other station-level variables
lines = lines.merge(station[['station_id', 'is_public', 'am_band']], how='inner', on='station_id')
lines = lines.merge(state[['state_id', 'census_region_5way']], how='inner', on='state_id')
lines.rename({'census_region_5way': 'station_census_region'}, axis=1, inplace=True)

lines['duration'] = lines['segment_end_global'] - lines['segment_start_global']

In [ ]:
## Assign snippet IDs
lines['snippet_id'] = snippet_id_start + np.arange(lines.shape[0])

In [ ]:
## Checks

assert lines['state_id'].notna().all()
assert lines['state_id'].isin(state['state_id']).all()

assert lines['show_id'].notna().all()
assert lines['show_id'].isin(show['show_id']).all()

assert lines['station_id'].notna().all()
assert lines['station_id'].isin(station['station_id']).all()

In [ ]:
lines.shape

In [ ]:
lines.head()

# Process

In [ ]:
radio_episode = pd.concat([
    lines.groupby(['date', 'show_id', 'station_id']).size().rename('cnt'),
    lines.groupby(['date', 'show_id', 'station_id'])['duration'].sum().rename('duration'),
    lines.groupby(['date', 'show_id', 'station_id'])['show_confidence'].mean().rename('show_confidence'),
    lines.groupby(['date', 'show_id', 'station_id'])['mean_word_confidence'].mean().rename('word_confidence'),
], axis=1).reset_index()

assert radio_episode['date'].notna().all()
assert radio_episode['show_id'].notna().all()
assert radio_episode['station_id'].notna().all()

In [ ]:
radio_best_episode_confidence_only = pysqldf('''
select
    x.date,
    x.show_id,
    x.station_id
from
(
    select
        te.date,
        te.show_id,
        te.station_id,

        row_number() over (
            partition by te.date, te.show_id
            order by
                te.show_confidence desc,
                te.cnt desc,
                te.word_confidence desc
        ) as quality
    from radio_episode te
    where
        te.word_confidence >= 0.85 and
        te.show_confidence >= 0.7
) x
where
    x.quality = 1;
''')

assert radio_best_episode_confidence_only['date'].notna().all()
assert radio_best_episode_confidence_only['show_id'].notna().all()
assert radio_best_episode_confidence_only['station_id'].notna().all()

In [ ]:
%%time

out = lines.merge(
    radio_best_episode_confidence_only,
    how='inner',
    on=['date', 'show_id', 'station_id']
)

out.rename({
    'start_dt': 'timestamp',
    'segment_idx': 'audio_file_index',
    'segment_start_relative': 'audio_file_offset',
}, axis=1, inplace=True)

out = out[['snippet_id', 'timestamp', 'end_dt', 'date', 'year', 'show_id', 'station_id',
           'audio_key', 'audio_file_offset', 'audio_file_index', 'duration',
           'station_census_region', 'is_public', 'am_band', 'content']]

assert out['date'].notna().all()
assert out['show_id'].notna().all()
assert out['station_id'].notna().all()

In [ ]:
out.shape

In [ ]:
out.groupby('year').size()

In [ ]:
out.head(10)

# Save

In [ ]:
with gzip.open('data/paper-round-3/radio/new-data-processed.csv.gz', 'wt') as f:
    out.to_csv(f, index=False)